## Federated classifier

In [1]:
import time
import pickle
import numpy as np
from tqdm import tqdm
from tensorboardX import SummaryWriter
import torch
import torch.nn as nn
import torch.optim as optim
import copy
import torch.nn.functional as F
import pandas as pd
from torch.autograd import Variable
from sklearn.preprocessing import StandardScaler
from torch.autograd import Variable
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from torch.utils.data import TensorDataset
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix


In [2]:
# This function takes a list of model weights (each model weight being a dictionary), averages them and returns the average weights
def average_weights(w):
    """
    Returns the average of the weights.
    """
    w_avg = copy.deepcopy(w[0])
    for key in w_avg.keys():
        for i in range(1, len(w)):
            w_avg[key] += w[i][key]
        w_avg[key] = torch.div(w_avg[key], len(w))
    return w_avg



#This function splits a dataset into a number of independent and identically distributed (IID) subsets. It is done in such a way that each user or client receives a unique set of samples.

def split_iid(dataset, num_users):
    num_items = int(len(dataset)/num_users) # the number of allocated samples for each client
    dict_users, all_idxs = {}, [i for i in range (len(dataset))]
    for i in range (num_users):
        dict_users[i] = set(np.random.choice(all_idxs, num_items, replace=False))
        all_idxs = list (set(all_idxs) - dict_users[i])
    return dict_users 


# This function reads the training and test data from a csv file, 
#performs some preprocessing operations and returns a TensorDataset for both training and test data. 
#It also splits the training data into multiple subsets using the unlabeled_iid() function and returns these user groups
def get_dataset(options):
    
    
    #print("x_shape", x.shape)
    
    file_labeled_train = pd.read_csv('IIoT_edge_base_classifier_pred_new.csv', skiprows=0, sep=',')
    x_train=file_labeled_train.values[:, 0:30]
    y_train=file_labeled_train.values[:, 30]
    #y_train=pd.get_dummies(y_train)
    #y_train = y_train.astype(np.float32)
    #print(y_train.dtypes)
    #y_train = y_train.to_numpy()
    print("y_train_shape", y_train.shape)
    print("y_train", y_train)
    #y_class=len(set(y_train))
    #print("y_class", y_class)

    
    file_labeled_test = pd.read_csv('IIoT_edge_base_classifier_pred_test_new.csv', skiprows=0, sep=',')
    x_test=file_labeled_test.values[:, 0:30]
    y_test=file_labeled_test.values[:, 30]
    #y_test=pd.get_dummies(y_test)
    #print(y_test.dtypes)
    #y_test = y_test.astype(np.float32)
    #y_test = y_test.to_numpy()
    print("y_test_shape", y_test.shape)
    print("y_test", y_test)
    #y_class_test=len(set(y_test))
    #print("y_class_test", y_class_test)
    
    #raise
    
    #train_data
    #x_train=pd.DataFrame(x_test)
    #y_train=pd.DataFrame(y)

    #train_data=pd.concat([x_train, y_train],axis=1)
    #print("train_data", train_data.shape)
    
    #train_data.to_csv('IIoT_edge_base_classifier_pred_dummies.csv',index=False)

    #test_data
    #x_test=pd.DataFrame(x_test)
    #y_test=pd.DataFrame(y_test)

    #test_data=pd.concat([x_val, y_val], axis=1)
    #print("test_data", test_data.shape)
    
    #train_data.to_csv('IIoT_edge_base_classifier_pred_test_dummies.csv',index=False)

    
    #x_train_l,x_test,y_train_l,y_test=train_test_split(x_l, y ,random_state=0, test_size=0.5)
    
    #print(x_train_l.dtype) # This should not be 'object'
    
    #train_data = train_data.astype(float)
    #test_data = test_data.astype(float)
    #y_train_l = y_train_l.astype(float)
    #y_test=y_test.astype(float)

    #sc = MinMaxScaler()
    #x_train_l = sc.fit_transform(x_train_l)
    #x_test_l = sc.transform(x_test_l)
    
    #scaler=StandardScaler()
    #x_train_l=scaler.fit_transform(x_train_l)
    #x_test=scaler.fit_transform(x_test)
    
    #scaler=StandardScaler()
    #x_train=scaler.fit_transform(x_train)
    #x_test=scaler.fit_transform(x_test)


    
    train_dataset = TensorDataset(torch.from_numpy(x_train),torch.from_numpy(y_train))   
    test_dataset = TensorDataset(torch.from_numpy(x_test),torch.from_numpy(y_test))   

    
    
    #train_dataset = TensorDataset(torch.from_numpy(train_data))
    #test_dataset = TensorDataset(torch.from_numpy(test_data))

    
    #sc = MinMaxScaler()
    #unlabeled_dataset = sc.fit_transform(x)

    
    
    #Feature scaling
    #sc = StandardScaler()
    #unlabeled_dataset = sc.fit_transform(x)

    #converting to torch tensor
    #train_dataset = torch.tensor(train_dataset, dtype=torch.float32)
    #test_dataset = torch.tensor(test_dataset, dtype=torch.float32)

    #print("train_dataset")

    
    user_groups = split_iid(train_dataset, options.num_users)
    print("user_group", user_groups)
    print("Done...")
    return train_dataset, test_dataset, user_groups




#This class is a custom dataset that receives a base dataset and a list of indices. 
#It enables getting only a subset of the data in the base dataset, as specified by the indices list. 
#This is helpful in a federated learning scenario, where each client might only have access to a subset of the total data.
class DatasetSplit(Dataset):
    def __init__(self, dataset, idxs):
        self.dataset = dataset
        self.idxs = [int(i) for i in idxs]
    def __len__(self):
        return len(self.idxs)

    def __getitem__(self, item):
        
        data, target = self.dataset[self.idxs[item]]
        return torch.tensor(data), torch.tensor(target)

        #data = self.dataset[self.idxs[item]]
        #return torch.tensor(data), torch.tensor(data)

In [3]:
#set program arguments
options = lambda: None
options.rounds= 10
options.num_users= 15
options.frac= 1
options.local_ep= 10
options.local_bs= 10
options.lr= 0.01
options.momentum= 0.5
options.model= "mlp"
options.kernel_num= 9
options.kernel_sizes= "3,4,5"
options.num_channels= 1
options.norm= "batch_norm"
options.num_filters= 32
options.max_pool= "True"
options.dataset= "mnist"
options.num_classes= 10
options.gpu= None
options.optimizer= "sgd"
options.iid= 1
options.unequal= 0
options.stopping_rounds= 10
options.verbose= 1
options.seed= 1
options.batch_size = 100
options.client_epochs = 50
options.server_epochs=20



In [4]:
input_size = n_features=30
hidden_size = [100, 100, 15]

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.MLP= nn.Sequential(
        nn.Linear(input_size, hidden_size[0]),
        nn.ReLU(),
        nn.Dropout(0.01),
        nn.Linear(hidden_size[0], hidden_size[1]),
        nn.ReLU(),
        nn.Dropout(0.01),
        nn.Linear(hidden_size[1], hidden_size[2]))
        #nn.Softmax())

    def forward(self, x):
        x=self.MLP(x)
        return x
    

In [5]:
class MLPModel(object):
    def __init__(self, options, train_dataset, test_dataset, idxs, logger):
        
        self.options = options
        self.logger = logger
        self.train_loader = DataLoader(DatasetSplit(train_dataset, idxs),
                             batch_size=options.batch_size, shuffle=True)
        self.test_loader= DataLoader(test_dataset, batch_size= options.batch_size, shuffle=False)

        #size=sys.getsizeof(self.train_loader)
        #print("data_user", size)
        self.device = 'cuda' if options.gpu else 'cpu'
        # Default criterion set to NLL loss function
        #self.criterion = nn.NLLLoss()
        self.criterion = nn.CrossEntropyLoss()
        #self.criterion= nn.BCELoss()
        self.client_epochs=options.client_epochs
        self.net=MLP().to(device)
        self.optimizer = optim.Adam(self.net.parameters(), lr=0.01)

        self.history = {}
        self.history['train_loss'] = []
        self.history['test_loss'] = []

    def train(self, model):
        mean_losses_superv = []
        #self.net.train()
        total = 0
        correct = 0
        for epoch in range(self.options.client_epochs):
            h = np.array([])
            
            for x, y in self.train_loader:
                
                self.optimizer.zero_grad()

                x=x.float()
                
                output = self.net(x)
                
                y=y.long()
                
                loss = self.criterion(output, y)
                h = np.append(h, loss.item())
                #raise
                
        # ===================backward====================
                loss.backward()
                self.optimizer.step()
                output = output.argmax(axis=1)
                
                total += y.size(0)
                
                y=y.float()
                output=output.float()
                
                correct += (output == y).sum().item()
                
        #raise
        # ===================log========================
            mean_loss_superv = np.mean(h)
            train_acc=correct/total
            
            mean_losses_superv.append(mean_loss_superv)
            PATH = "state_dict_model_Sup_IIoT_edge.pt"
            # Save
            torch.save(self.net.state_dict(), PATH)
            return sum(mean_losses_superv) / len(mean_losses_superv) , train_acc, self.net.state_dict() 
            #print('Done.....')

   
    def test_inference(self, model, test_dataset):
        nb_classes = 15
        confusion_matrix = np.zeros((nb_classes, nb_classes))
        self.net.eval()
        test_loss = 0
        correct = 0
        total = 0
        output_list=torch.zeros(0,dtype=torch.long)
        target_list=torch.zeros(0,dtype=torch.long)
        with torch.no_grad():
            
            for data, target in self.test_loader:
                
                data, target = data.to(device), target.to(device)
        
                
                output = self.net(data.float())
                
                batch_loss = self.criterion(output, target.long())
                #print("done... test...")
                #raise
                test_loss += batch_loss.item()
                total += target.size(0)
                
                

                target=target.float()
                
                output=output.argmax(axis=1)
                output=output.float()
                
                
                output_list=torch.cat([output_list, output.view(-1).long()])
                target_list=torch.cat([target_list, target.view(-1).long()])
                
        
                correct += (output == target).sum().item()
        
                    
            test_loss/=total
            acc=correct/total
       
    
            F1_score= f1_score(target_list, output_list, average = "macro") #labels=np.unique(output_list))))
            Precision=precision_score(target_list, output_list, average="macro")                                      
            Recall=recall_score(target_list, output_list, average="macro")
            class_report=classification_report(target_list,output_list, digits=4)

            #print(' F1 Score : ' + str(f1_score(target_list, output_list, average = "macro"))) #labels=np.unique(output_list))))
            #print(' Presicion : '+str(precision_score(target_list, output_list, average="macro", labels=np.unique(output_list))))                                       
            #print(' Recall : '+str(recall_score(target_list, output_list, average="macro", labels=np.unique(output_list))))
            #print("report", classification_report(target_list,output_list, digits=4))

            return acc, F1_score, Precision, Recall, class_report, test_loss



In [6]:
logger = SummaryWriter('../logs')
#load data
train_dataset, test_dataset,user_groups=get_dataset(options)


y_train_shape (24510,)
y_train [ 8.  0.  4. ...  7.  2. 10.]
y_test_shape (7004,)
y_test [ 2. 10. 10. ...  3. 10.  3.]
user_group {0: {16390, 6, 16398, 8206, 23, 8218, 16412, 31, 16418, 8233, 16426, 46, 16432, 53, 55, 61, 8255, 8257, 8259, 16464, 16465, 8286, 16478, 16481, 16491, 8302, 16500, 116, 8313, 123, 133, 138, 8333, 16530, 16532, 16543, 162, 172, 16563, 16571, 191, 16578, 195, 8389, 8392, 8393, 8394, 8398, 16593, 16598, 8407, 219, 8412, 8416, 8424, 16628, 247, 248, 16633, 8191, 8444, 16639, 8452, 261, 8453, 16647, 264, 288, 8489, 16684, 8496, 16688, 309, 314, 317, 16707, 16713, 16716, 339, 8536, 16729, 8557, 16767, 388, 395, 8592, 410, 8626, 16819, 8632, 8641, 16837, 456, 464, 16849, 470, 480, 8680, 16882, 16898, 521, 16908, 16914, 8727, 8735, 550, 558, 8759, 16952, 16955, 16958, 8770, 16963, 8774, 587, 16972, 8781, 600, 8803, 16997, 616, 8808, 17006, 624, 625, 8817, 627, 8819, 629, 8824, 8826, 8827, 638, 639, 17028, 649, 17034, 651, 8844, 654, 8857, 8858, 8863, 17059, 8870, 68

In [13]:

# BUILD MODEL
#global_model=MLPModel(options=options, train_dataset=train_dataset, test_dataset=test_dataset, logger=logger)
MLP_model=MLP()
print(MLP_model)
# Training
train_loss, train_accuracy = [], []
val_acc_list, net_list = [], []
cv_loss, cv_acc = [], []
print_every = 2
val_loss_pre, counter = 0, 0

for rounds in tqdm(range(options.rounds)):
    # in the server
    local_weights, local_losses = [], []
    print(f'\n | Training Round : {rounds+1} |\n')
    
    #global_model.train(auto_encoder_model)
    m = max(int(options.frac * options.num_users), 1)
    idxs_users = np.random.choice(range(options.num_users), m, replace=False)
    print("idxs_users",idxs_users)

    
    
    for idx in idxs_users:
        MLP_client = MLPModel(options=options, train_dataset=train_dataset, test_dataset=test_dataset,
                                    idxs=user_groups[idx], logger=logger)
        loss, train_acc, w = MLP_client.train(model=copy.deepcopy(MLP_model))
        #print(w)
        #print("w", w)
        #print("loss", loss)
        local_weights.append(copy.deepcopy(w))
        #local_losses.append(copy.deepcopy(loss))
        
        test_acc, F1_score, Precision, Recall, class_report, test_loss = MLP_client.test_inference(MLP_model, test_dataset)
        print(f'client_id {idx}')
        print("|---- Test Accuracy_client: {:.2f}%".format(test_acc))
        print("|---- F1_score:", F1_score)
        print("|---- Precision:", Precision)
        print("|---- Recall:", Recall)
        print(class_report)
        print(f'Testing Loss : {np.mean(np.array(test_loss))}')
    #print(local_weights)
    MLP_model.load_state_dict(average_weights(local_weights))
    #loss_avg = sum(local_losses) / len(local_losses)
    #train_loss.append(loss_avg)
    
    #mean_losses_superv = global_model.train(auto_encoder_model)
    #mean_losses_superv_avg = sum(mean_losses_superv) / len(mean_losses_superv)
    
    
   
    # Calculate avg training accuracy over all users at every epoch
    #list_acc, list_loss = [], []
    #auto_encoder_model.eval()
    #for c in range(options.num_users):
        #local_model = AutoEncoder(options=options, dataset=train_dataset,
                                       #idxs=user_groups[idx], logger=logger)
        #acc, loss = auto_encoder.inference(model=auto_encoder_model)
        #list_acc.append(acc)
        #list_loss.append(loss)
    #train_accuracy.append(sum(list_acc)/len(list_acc))

    # print global training loss after every 'i' rounds
    #if (round+1) % print_every == 0:
        #print(f' \nAvg Training Stats after {round+1} global rounds:')
        #print(f'Training Loss : {np.mean(np.array(train_loss))}')
        #print('Train Accuracy: {:.2f}% \n'.format(100*train_accuracy[-1]))
#print("yes")

    test_acc_g, F1_score_g, Precision_g, Recall_g, class_report_g, test_loss_g = MLP_client.test_inference(MLP_model, test_dataset)

    
    #if (rounds+1) % print_every == 0:
    print(f' \nAvg Training Stats after {rounds+1} global rounds:')
    print("|---- Test Accuracy:", test_acc_g)
    print("|---- F1_score:", F1_score_g)
    print("|---- Precision:", Precision_g)
    print("|---- Recall:", Recall_g)
    print(class_report_g)
    #print(f'Training Loss : {np.mean(np.array(mean_losses_superv_avg))}')
    print(f'Testing Loss : {np.mean(np.array(test_loss))}')
    
   
        #print(f' \n Results after {options.rounds} global rounds of training:')
#print("|---- Avg Train Accuracy: {:.2f}%".format(100*train_accuracy[-1]))
        #print(f'Training Loss : {np.mean(np.array(mean_losses_superv_avg))}')
        #print(f'Testing Loss : {np.mean(np.array(test_loss))}')
        
        #print("yes")

  0%|          | 0/10 [00:00<?, ?it/s]

MLP(
  (MLP): Sequential(
    (0): Linear(in_features=30, out_features=100, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.01, inplace=False)
    (3): Linear(in_features=100, out_features=100, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.01, inplace=False)
    (6): Linear(in_features=100, out_features=15, bias=True)
  )
)

 | Training Round : 1 |

idxs_users [11  0  5 12  9  3  1  4 10 14  7 13  6  8  2]


C:\Users\perera\Anaconda3\lib\site-packages\ipykernel_launcher.py:135: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


client_id 11
|---- Test Accuracy_client: 0.87%
|---- F1_score: 0.7325097906328765
|---- Precision: 0.7724852801354437
|---- Recall: 0.7427906670591892
              precision    recall  f1-score   support

           0     1.0000    1.0000    1.0000      1145
           1     1.0000    1.0000    1.0000       668
           2     1.0000    1.0000    1.0000       583
           3     0.8648    0.9759    0.9170       498
           4     0.4729    0.9529    0.6321       467
           5     0.9785    0.2172    0.3555       419
           6     1.0000    0.6198    0.7653       455
           7     0.9105    0.9978    0.9522       459
           8     1.0000    0.9485    0.9736       466
           9     1.0000    0.9332    0.9654       449
          10     0.8911    0.9326    0.9114       430
          11     0.9381    1.0000    0.9681       455
          12     0.5313    0.5640    0.5472       406
          13     0.0000    0.0000    0.0000        57
          14     0.0000    0.0000    0

client_id 4
|---- Test Accuracy_client: 0.87%
|---- F1_score: 0.7325641628681636
|---- Precision: 0.7605320154078138
|---- Recall: 0.7431482854844587
              precision    recall  f1-score   support

           0     1.0000    1.0000    1.0000      1145
           1     1.0000    1.0000    1.0000       668
           2     1.0000    1.0000    1.0000       583
           3     0.8294    0.9759    0.8967       498
           4     0.4831    0.8244    0.6092       467
           5     0.9796    0.2291    0.3714       419
           6     0.7731    0.7341    0.7531       455
           7     0.9105    0.9978    0.9522       459
           8     1.0000    0.9485    0.9736       466
           9     1.0000    0.9310    0.9642       449
          10     0.9413    0.9326    0.9369       430
          11     0.9362    1.0000    0.9671       455
          12     0.5548    0.5739    0.5642       406
          13     0.0000    0.0000    0.0000        57
          14     0.0000    0.0000    0.

...

client_id 2
|---- Test Accuracy_client: 0.89%
|---- F1_score: 0.7560739029953472
|---- Precision: 0.7747061250412567
|---- Recall: 0.7610058184314414
              precision    recall  f1-score   support

           0     1.0000    1.0000    1.0000      1145
           1     1.0000    1.0000    1.0000       668
           2     1.0000    1.0000    1.0000       583
           3     0.8294    0.9759    0.8967       498
           4     0.5417    0.7645    0.6341       467
           5     0.5565    0.7757    0.6481       419
           6     0.9105    0.6484    0.7574       455
           7     0.9105    0.9978    0.9522       459
           8     1.0000    0.9485    0.9736       466
           9     1.0000    0.9310    0.9642       449
          10     0.9413    0.9326    0.9369       430
          11     0.9362    1.0000    0.9671       455
          12     0.9944    0.4409    0.6109       406
          13     0.0000    0.0000    0.0000        57
          14     0.0000    0.0000    0.

client_id 10
|---- Test Accuracy_client: 0.88%
|---- F1_score: 0.7423925546351409
|---- Precision: 0.7685807738883008
|---- Recall: 0.7509761280375826
              precision    recall  f1-score   support

           0     1.0000    1.0000    1.0000      1145
           1     1.0000    1.0000    1.0000       668
           2     1.0000    1.0000    1.0000       583
           3     0.9187    0.9759    0.9464       498
           4     0.7449    0.3940    0.5154       467
           5     0.4362    0.9308    0.5941       419
           6     0.7687    0.7231    0.7452       455
           7     0.9105    0.9978    0.9522       459
           8     1.0000    0.9485    0.9736       466
           9     1.0000    0.9310    0.9642       449
          10     0.8302    0.9326    0.8784       430
          11     0.9362    1.0000    0.9671       455
          12     0.9831    0.4310    0.5993       406
          13     0.0000    0.0000    0.0000        57
          14     0.0000    0.0000    0

client_id 0
|---- Test Accuracy_client: 0.85%
|---- F1_score: 0.7195221005271223
|---- Precision: 0.7967859899889863
|---- Recall: 0.7279006938474748
              precision    recall  f1-score   support

           0     1.0000    1.0000    1.0000      1145
           1     1.0000    1.0000    1.0000       668
           2     1.0000    1.0000    1.0000       583
           3     0.9624    0.9759    0.9691       498
           4     1.0000    0.2484    0.3979       467
           5     0.9237    0.2601    0.4060       419
           6     1.0000    0.6198    0.7653       455
           7     0.8691    0.9978    0.9290       459
           8     1.0000    0.9485    0.9736       466
           9     1.0000    0.9354    0.9666       449
          10     0.9413    0.9326    0.9369       430
          11     0.9401    1.0000    0.9691       455
          12     0.3152    1.0000    0.4793       406
          13     0.0000    0.0000    0.0000        57
          14     0.0000    0.0000    0.

100%|██████████| 10/10 [00:54<00:00,  5.46s/it]

client_id 11
|---- Test Accuracy_client: 0.84%
|---- F1_score: 0.6913922770446332
|---- Precision: 0.7247035856818393
|---- Recall: 0.7126753484864342
              precision    recall  f1-score   support

           0     1.0000    1.0000    1.0000      1145
           1     0.9214    1.0000    0.9591       668
           2     1.0000    1.0000    1.0000       583
           3     0.9624    0.9759    0.9691       498
           4     1.0000    0.3041    0.4663       467
           5     0.0000    0.0000    0.0000       419
           6     0.9728    0.6286    0.7637       455
           7     0.9105    0.9978    0.9522       459
           8     1.0000    0.9485    0.9736       466
           9     0.9470    0.9555    0.9512       449
          10     0.8911    0.9326    0.9114       430
          11     0.9557    0.9473    0.9514       455
          12     0.3097    1.0000    0.4729       406
          13     0.0000    0.0000    0.0000        57
          14     0.0000    0.0000    0